## СОДЕРЖАНИЕ:
* [Подготовка](#first-bullet)
* [Способ TF-IDF](#sec-bullet)
* [Способ BERT](#tree-bullet)
* [Выводы](#four-bullet)

# Проект для «Викишоп» с BERT (и TF-IDF)

Нужено создать инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

In [1]:
import os
import numpy as np
import pandas as pd
import spacy
import torch
import transformers
from tqdm import notebook
from tqdm.notebook import tqdm
tqdm.pandas()
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
#import nltk
#from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
#from catboost import CatBoostClassifier
import re
#from nltk.tokenize import word_tokenize


In [2]:
STATE = 12345

# Подготовка <a class="anchor" id="first-bullet"></a>

In [3]:
#чтение датасета
pth1 = './toxic_comments.csv'
pth2 = '/datasets/toxic_comments.csv'

if os.path.exists(pth1):
    comments = pd.read_csv(pth1)
elif os.path.exists(pth2):
    comments = pd.read_csv(pth2)
else:
    print('Something is wrong, что-то пошло не так как надо')
    

In [4]:
comments = comments.drop('Unnamed: 0', axis=1)

In [5]:
comments.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [6]:
comments.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 79.9 MB


In [7]:
comments['toxic'].mean()

0.10161213369158527

балланс слабый, всего 10% класса 1

###### Выводы

Данные довольно обьемные, почти 160 тыс строк и всего два столбца.

С первых же строк видно что текст в комментариях не очищен и не лемматизирован, то есть "сырой".

Распределение целевого признака сильно неравномерное. "Токсичных" комментариев всего 10% от общей массы.

# Способ TF-IDF <a class="anchor" id="sec-bullet"></a>

очистка и лемматизация текста

In [ ]:
#функция для приведения к нижнему регистру, удаление https com ссылок, и лемматизация библиотекой nltk
def clear_lemma_text(text):
    summ = []
    text = re.sub("[^a-zA-Z]"," ",text).lower()
    text = re.sub(r'http\S+', '', text) #remove_https
    text = re.sub(r"\ [A-Za-z]*.com", " ", text) #remove_com
    text = text.strip() #убирание первого и последнего пробела
    text = re.sub(r' {1,10}', ' ', text) #замена множественных пробелов на один
    
    for j in text.split():
        #print(j)
        if not j in stopwords:
            summ.append(lemmatize.lemmatize(j))
            
    
    text = ' '.join(summ)
    
    
    return text

In [ ]:
#инициализируем модель spacy 'en' 
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

In [ ]:
#функция очистки, удаление https com ссылок, и лемматизация библиотекой spacy
def clear_lemma_spacy_text(text):
    summ = []
    text = re.sub("[^a-zA-Z]"," ",text).lower()
    text = re.sub(r'http\S+', '', text) #remove_https
    text = re.sub(r"\ [A-Za-z]*.com", " ", text) #remove_com
    text = text.strip() #убирание первого и последнего пробела
    text = re.sub(r' {1,10}', ' ', text) #замена множественных пробелов на один
    
    doc = nlp(text) #лемматизация spacy
    text = " ".join([token.lemma_ for token in doc]) #сборка предложения обратно
    
    
    return text

In [ ]:
#проверка
t = comments['text'][33]
t

In [ ]:
#проверка
spacy = clear_lemma_spacy_text(t)
spacy

код ниже пришлось сделать так как лемматизация выполняется долго и легче сделать её один раз и потом скачать с диска готовый датасет

In [ ]:
#чтение лемматизированного датасета
pth1 = './toxic_comments_lemma.csv'

if os.path.exists(pth1):
    comments = pd.read_csv(pth1)
else:
    print('Если выпала эта надпись, надо раскомментировать две строки снизу')
    

In [ ]:
#должны быть 3 колонки `text` `toxic` `text_lemma`
comments.head()

## разделение на выборки

In [ ]:
#на тестовую выборку отделю 10%, так как данных много
x_train, x_test, y_train, y_test = train_test_split(
        comments['text_lemma'], 
        comments['toxic'], 
        test_size=0.1, 
    stratify = comments['toxic'], 
        random_state=STATE)

In [ ]:
#проверка размера получившихся выборок
x_train.shape, x_test.shape, y_train.shape, y_test.shape

In [ ]:
#проверка балланса целевого прикзнака в выборках
y_train.mean(), y_test.mean()

## обработка по методу TF_IDF

In [ ]:
#определяем счетчик слов
count_tf_idf = TfidfVectorizer(max_df = 0.9,
                               min_df = 4e-4, #ограничение по встречаемости слов снизу, всё что меньше - за борт
                               stop_words='english')

In [ ]:
#обучение и преобразование тренировочной выборки
tf_idf_train = count_tf_idf.fit_transform(x_train)

In [ ]:
#проверка размера тренировочной выборки после обработки TF-IDF
tf_idf_train.shape

In [ ]:
#минимальное значение в выборке (из любопытсва)
tf_idf_train.min()

In [ ]:
#среднее значение в выборке
tf_idf_train.mean()

In [ ]:
#максимальное значение в выборке
tf_idf_train.max()

In [ ]:
tf_idf_test = count_tf_idf.transform(x_test)

## Обучение (TF_IDF)

#### LogisticRegression и GridSearchCV (TF-IDF)

In [ ]:
params = {'C':[1,10], 
          'solver':['sag'], 
          'max_iter':[100,500],
          'class_weight':['balanced', None]
         } 

In [ ]:
lr = LogisticRegression(random_state=STATE)

In [ ]:
clf = GridSearchCV(lr,param_grid=params , cv=3, scoring='f1', n_jobs=-1)

In [ ]:
%%time
clf.fit(tf_idf_train, y_train)

In [ ]:
#лучший результат
clf.best_score_

In [ ]:
#полная таблица с результатами перебора
grid_res = pd.DataFrame(clf.cv_results_).sort_values(by='mean_test_score', ascending=False)
grid_res

In [ ]:
#заново обучаем модель с лучшими гиперпараметрами
lr = LogisticRegression(C=10, solver='sag', max_iter=100)

In [ ]:
%%time
lr.fit(tf_idf_train, y_train)

###### Вывод

лучший результат f1=0.768

Использования балланса в настройках модели ухудшает результат.

#### RandomForestClassifier и GridSearchCV (TF-IDF)

In [ ]:
params = {'n_estimators':[40,60], 
          'max_depth':[120,160], 
          'min_samples_split':[2], 
          'min_samples_leaf':[2], 
          'class_weight':['balanced', None]
         } 

In [ ]:
rf = RandomForestClassifier(random_state=STATE)

In [ ]:
clf = GridSearchCV(rf,param_grid=params , cv=3, scoring='f1', n_jobs=-1)

In [ ]:
%%time
clf.fit(tf_idf_train, y_train)

In [ ]:
#лучший результат
clf.best_score_

In [ ]:
#лучший результат
clf.best_params_

In [ ]:
#полная таблица с результатами перебора
grid_res = pd.DataFrame(clf.cv_results_).sort_values(by='mean_test_score', ascending=False)
grid_res

###### Вывод

лучший результат f1=0.737

#### CatBoostClassifier и GridSearchCV (TF-IDF)

In [ ]:
grid = {'learning_rate': [0.5],
        'depth': [6],
        
        #'l2_leaf_reg': [1]
       }

In [ ]:
CatBoost = CatBoostClassifier(eval_metric="F1:hints=skip_train~false", iterations=50)

In [ ]:
grid_search_result = CatBoost.grid_search(grid,
                                          X=tf_idf_train,
                                          y=y_train, 
                                          cv=3,
                                          calc_cv_statistics=True,
                                          search_by_train_test_split=True,
                                          refit=True,
                                          partition_random_seed=STATE, 
                                          plot=False)

In [ ]:
CatBoost.best_score_

###### Вывод

лучший результат f1=0.753

Наверняка можно получить лучше результат, но и так этот перебор параметров выполняется слишком долго, поэтому я оставил как есть.

## Тест лучшей модели (TF_IDF)

Лучшей моделью из трёх оказалась LogisticRegression. Она же и самая быстрая. Все сотальные модели требуют много времени на обучение, возможно они бы оказались лучше.

Вычислю метрику F1 для неё на тестовой выборке

In [ ]:
pred_test = lr.predict(tf_idf_test)

In [ ]:
f1_score(y_test, pred_test)

###### Вывод

результат на тестовой выборке для лучшей модели LogisticRegression f1=0.78

# Способ BERT <a class="anchor" id="tree-bullet"></a>

#### Классификация на эмбеддингах

In [8]:
comments_bert = comments.sample(n=10000, random_state=STATE) # отбор от датасета 10 тыс случайных строк

In [9]:
#проверка что балланс целевого признака не изменился
comments_bert.toxic.mean()

0.1018

In [10]:
#сброс индексов
comments_bert = comments_bert.reset_index(drop=True)

In [11]:
comments_bert.head(10)

,text,toxic
0,Expert Categorizers \n\nWhy is there no menti...,0
1,"""\n\n Noise \n\nfart* talk. """,1
2,"An indefinite block is appropriate, even for a...",0
3,I don't understand why we have a screenshot of...,0
4,"Hello! Some of the people, places or things yo...",0
5,"""::::::::::::::If you read carefully, my comme...",1
6,"""\nDid you make it at WP:AE? ofShalott """,0
7,know as much as he thinks.,0
8,Thank you for your comment and willingness to ...,0
9,UPDATE 3-12-06\n\nHEY YALL IM GOING TO BE GONE...,0


In [12]:
#токенизация с unitary/toxic-bert
tokenizer = transformers.BertTokenizer.from_pretrained('unitary/toxic-bert')

tokenized = comments_bert['text'].apply(
    lambda x: tokenizer.encode(x, add_special_tokens=True,  truncation=True, max_length=512))

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)

In [13]:
tokenized.shape

(10000,)

In [14]:
padded.shape

(10000, 512)

In [15]:
attention_mask.shape

(10000, 512)

In [16]:
#количество строк дла окончательной прогонки модели bert
N_bert = 400

In [17]:
#отбор N случайных элементов (N_bert)
np.random.seed(STATE) #зафиксировать случайность, для воспроизводимости результата.
idx_N = np.random.randint(padded.shape[0], size=N_bert)

padded_N = padded[idx_N,:]

In [18]:
#проверка размера
padded_N.shape

(400, 512)

In [19]:
model = transformers.BertModel.from_pretrained('unitary/toxic-bert')

Some weights of the model checkpoint at unitary/toxic-bert were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [20]:
batch_size = 100
embeddings = []
for i in notebook.tqdm(range(padded_N.shape[0] // batch_size)):
        batch = torch.LongTensor(padded_N[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())

  0%|          | 0/4 [00:00<?, ?it/s]

In [21]:
features = np.concatenate(embeddings)

In [22]:
features.shape

(400, 768)

In [39]:
#чтение с диска
features_pd = pd.read_csv('./features_bert.csv')
features = features_pd.to_numpy()
features.shape

(400, 768)

#### отбор целевого признака по полученным случайным индексам `idx_N`.

In [132]:
y = comments_bert['toxic'].iloc[idx_N]
len(y)

400

In [42]:
y = y.reset_index(drop=True).to_numpy()

In [43]:
#проверка
y[:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [44]:
#проверка
idx_N[:10]

array([4578, 2177, 3492, 4094, 4478,  546, 7709, 3441, 7483, 6798])

#### делим выборку `comments_bert` на тренировочную и обучающую, в соотношении 50/50

(количество векторов в ней равно N_bert)

In [47]:
x_train, x_test, y_train, y_test = train_test_split(features, 
                                                    y, 
                                                    test_size=0.5, 
                                                    stratify = y,
                                                    random_state=STATE)

In [48]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((200, 768), (200, 768), (200,), (200,))

In [49]:
y_train.mean(), y_test.mean()

(0.105, 0.105)

#### LogisticRegression GridSearchCV (BERT)

In [96]:
params = {'C':[0.001,0.01,0.1], 
          'solver':['sag'], 
          'max_iter':[5,10,50],
          'class_weight':['balanced', None]
         }

In [97]:
lr = LogisticRegression(random_state=STATE)

In [98]:
clf = GridSearchCV(lr,param_grid=params , cv=3, scoring='f1', n_jobs=-1)

In [99]:
%%time
clf.fit(x_train, y_train)

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means 

CPU times: user 153 ms, sys: 11.6 ms, total: 164 ms
Wall time: 2.53 s


/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


GridSearchCV(cv=3, estimator=LogisticRegression(random_state=12345), n_jobs=-1,
             param_grid={'C': [0.001, 0.01, 0.1],
                         'class_weight': ['balanced', None],
                         'max_iter': [5, 10, 50], 'solver': ['sag']},
             scoring='f1')

In [100]:
#лучший результат
clf.best_score_

0.702020202020202

In [101]:
#лучшие параметры
clf.best_params_

{'C': 0.01, 'class_weight': None, 'max_iter': 5, 'solver': 'sag'}

In [ ]:
#лучшая модель
clf.best_estimator_

In [82]:
#полная таблица с результатами перебора
grid_res = pd.DataFrame(clf.cv_results_).sort_values(by='mean_test_score', ascending=False)
grid_res

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_class_weight,param_max_iter,param_solver,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
2,0.255408,0.021906,0.007126,0.003162,1,None,100,sag,"{'C': 1, 'class_weight': None, 'max_iter': 100...",0.545455,0.833333,0.666667,0.681818,0.118013,1
3,0.713220,0.062493,0.005616,0.001911,1,None,500,sag,"{'C': 1, 'class_weight': None, 'max_iter': 500...",0.545455,0.833333,0.666667,0.681818,0.118013,1
6,0.254546,0.022862,0.004472,0.000090,10,None,100,sag,"{'C': 10, 'class_weight': None, 'max_iter': 10...",0.545455,0.833333,0.666667,0.681818,0.118013,1
1,0.637094,0.093828,0.005239,0.001416,1,balanced,500,sag,"{'C': 1, 'class_weight': 'balanced', 'max_iter...",0.571429,0.857143,0.615385,0.681319,0.125615,4
7,1.076554,0.051317,0.004207,0.000076,10,None,500,sag,"{'C': 10, 'class_weight': None, 'max_iter': 50...",0.500000,0.833333,0.666667,0.666667,0.136083,5
4,0.225450,0.028099,0.004343,0.000126,10,balanced,100,sag,"{'C': 10, 'class_weight': 'balanced', 'max_ite...",0.461538,0.857143,0.615385,0.644689,0.162829,6
5,1.258236,0.037645,0.004313,0.000026,10,balanced,500,sag,"{'C': 10, 'class_weight': 'balanced', 'max_ite...",0.461538,0.833333,0.615385,0.636752,0.152535,7
0,0.319442,0.030834,0.010995,0.004701,1,balanced,100,sag,"{'C': 1, 'class_weight': 'balanced', 'max_iter...",0.461538,0.857143,0.571429,0.630037,0.166737,8


In [103]:
#заново обучаем модель с лучшими гиперпараметрами
lr = LogisticRegression(C=0.01, solver='sag', max_iter=5)
lr.fit(x_train, y_train)

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LogisticRegression(C=0.01, max_iter=5, solver='sag')

#### Вывод
лучший результат f1=0.702

#### RandomForestClassifier  и GridSearchCV (BERT)

In [116]:
params = {'n_estimators':[20,40,60], 
          'max_depth':[20,40,80], 
          'min_samples_split':[2,4,6], 
          'min_samples_leaf':[2,4,6], 
          #'class_weight':['balanced', None]
         } 

In [117]:
rf = RandomForestClassifier(random_state=STATE)

In [118]:
clf = GridSearchCV(rf,param_grid=params , cv=3, scoring='f1', n_jobs=-1)

In [119]:
%%time
clf.fit(x_train, y_train)

CPU times: user 680 ms, sys: 51.8 ms, total: 732 ms
Wall time: 33.8 s


GridSearchCV(cv=3, estimator=RandomForestClassifier(random_state=12345),
             n_jobs=-1,
             param_grid={'max_depth': [20, 40, 80],
                         'min_samples_leaf': [2, 4, 6],
                         'min_samples_split': [2, 4, 6],
                         'n_estimators': [20, 40, 60]},
             scoring='f1')

In [120]:
#лучший результат
clf.best_score_

0.702020202020202

In [121]:
#лучшие параметры
clf.best_params_

{'max_depth': 20,
 'min_samples_leaf': 2,
 'min_samples_split': 2,
 'n_estimators': 40}

In [125]:
#лучшая модель
clf.best_estimator_

RandomForestClassifier(max_depth=20, min_samples_leaf=2, n_estimators=40,
                       random_state=12345)

In [122]:
#полная таблица с результатами перебора
grid_res = pd.DataFrame(clf.cv_results_).sort_values(by='mean_test_score', ascending=False)
grid_res

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_min_samples_leaf,param_min_samples_split,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
34,0.241204,0.003487,0.022134,0.000576,40,2,6,40,"{'max_depth': 40, 'min_samples_leaf': 2, 'min_...",0.545455,0.833333,0.727273,0.702020,0.118875,1
28,0.219645,0.016622,0.028543,0.009415,40,2,2,40,"{'max_depth': 40, 'min_samples_leaf': 2, 'min_...",0.545455,0.833333,0.727273,0.702020,0.118875,1
58,0.285614,0.080015,0.039608,0.015444,80,2,4,40,"{'max_depth': 80, 'min_samples_leaf': 2, 'min_...",0.545455,0.833333,0.727273,0.702020,0.118875,1
29,0.369669,0.051700,0.033519,0.001710,40,2,2,60,"{'max_depth': 40, 'min_samples_leaf': 2, 'min_...",0.545455,0.833333,0.727273,0.702020,0.118875,1
62,0.524836,0.070013,0.057998,0.018860,80,2,6,60,"{'max_depth': 80, 'min_samples_leaf': 2, 'min_...",0.545455,0.833333,0.727273,0.702020,0.118875,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54,0.144355,0.015115,0.016241,0.001851,80,2,2,20,"{'max_depth': 80, 'min_samples_leaf': 2, 'min_...",0.545455,0.615385,0.666667,0.609169,0.049679,76
30,0.106897,0.006016,0.017137,0.002343,40,2,4,20,"{'max_depth': 40, 'min_samples_leaf': 2, 'min_...",0.545455,0.615385,0.666667,0.609169,0.049679,76
57,0.133929,0.033920,0.013941,0.000490,80,2,4,20,"{'max_depth': 80, 'min_samples_leaf': 2, 'min_...",0.545455,0.615385,0.666667,0.609169,0.049679,76
3,0.162069,0.027079,0.017426,0.003728,20,2,4,20,"{'max_depth': 20, 'min_samples_leaf': 2, 'min_...",0.545455,0.615385,0.666667,0.609169,0.049679,76


#### Вывод

лучший результат f1=0.702

#### CatBoostClassifier и GridSearchCV (BERT)

#### тестирование лучшей модели (BERT)

Модели LogisticRegression и RandomForestClassifier выдают одинаковый результат, но первая работает намного быстрее.

Поэтому для окончательной проверки выберу её.

In [129]:
pred_test = lr.predict(x_test)

In [130]:
f1_score(y_test, pred_test)

0.6285714285714286

#### Вывод

лучший результат f1=0.629 на тестовой выборке

# Выводы <a class="anchor" id="four-bullet"></a>

Проведена проверка двух типов обработки текста:
   - методика TF-IDF
   - языковая модель BERT
   
Рассмотрены 3 разные ML модели: LogisticRegression, RandomForestClassifier и CatBoost.

Точность f1, по методу TF-IDF,  для лучшей модели LogisticRegression составила f1=0.78

Точность для языковой модели BERT составила около 0.628 при количестве комментариев 200 для обучения и 200 для тестирования.